In [ ]:
## GIT
# Hacked tor restarting, session creating instead of using huge nested with statements.
# Rebuilt getPages method with new method.
# Added looping to get all the urls.
# Added parsing of the data.

In [ ]:
# TODO: Replace prints with logging.

In [278]:
import pandas as pd
import numpy as np

import re
import string

In [82]:
import requests
from torpy.http.requests import tor_requests_session
from selenium import webdriver
from bs4 import BeautifulSoup


In [83]:
import time
from fake_useragent import UserAgent

In [84]:
from subprocess import Popen

In [67]:
import os

def restart_tor():
    '''
    A forceful hack since tor doesn't function as intended on my windows and none of the solution work. Too bad! https://www.youtube.com/watch?v=k238XpMMn38
    
    Commented out code is how it should work.
    
    TODO: Descr.
    '''
    tor_path = 'C:/Users/Jurgis/Desktop/tor-win32-0.4.2.7/Tor/tor.exe'
    
    # Kill existing tor process.
    os.system("taskkill /F /im tor.exe")
    
    # Run a tor process in the background, wait for it to start.
    Popen(tor_path, shell=True)
    
    time.sleep(2)
    pass



# from stem import Signal
# from stem.control import Controller

# # signal TOR for a new connection 
# def renew_connection():
#     with Controller.from_port(port = 9051) as controller:
#         controller.authenticate(password="eatshit")
#         controller.signal(Signal.NEWNYM)


In [71]:
restart_tor()

In [90]:
def get_tor_session():
    # Use Tor ports for a requests.session.
    session = requests.session()
    session.proxies = {
        'http':  'socks5://127.0.0.1:9050',
        'https': 'socks5://127.0.0.1:9050'
    }
    
    # Set additional Tor session parameters.
    s.headers = UserAgent().random
    
    print('Built a Tor session at IP {}.'.format(session.get('http://httpbin.org/ip').json()['origin']))
    return session

# Make a request through the Tor connection
# IP visible through Tor
session = get_tor_session()
print(session.get("http://httpbin.org/ip").text)
# Above should print an IP different than your public IP

# Following prints your normal public IP
print(requests.get("http://httpbin.org/ip").text)

from stem import Signal
from stem.control import Controller

# signal TOR for a new connection 
restart_tor()
session = get_tor_session()
print(session.get("http://httpbin.org/ip").text)

Built a Tor session at IP 185.220.101.134.
{
  "origin": "185.220.101.134"
}

{
  "origin": "78.58.222.15"
}

Built a Tor session at IP 87.118.92.43.
{
  "origin": "87.118.92.43"
}



In [89]:
def get_number_of_pages(session, url):
    '''
    TODO: Descr.
    TODO: Handle SPECIFIC banned / captcha'ed cases with try excepts.
    '''
    try:
        # Set additional Tor session parameters.
        session.headers = UserAgent().random

        # Scrape the main page to get the total number of pages.
        page = session.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extract all page number buttons to find the maximum digit value.
        page_number_buttons = str(soup.find_all(class_='page-bt'))
        page_number_buttons = re.findall('\d+', page_number_buttons)

        total_pages = max([int(number) for number in page_number_buttons])

        return total_pages
    
    except Exception as e:
        return e

Built a Tor session at IP 185.220.101.134.


In [99]:
def get_page_urls(session, url):
    '''
    TODO: Descr.
    TODO: Handle SPECIFIC exceptions by restarting a tor session and re-running.
    '''
    try:
        # Get page contents and put them in a "soup".
        page = session.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Extract all listings and get their urls.
        listings = soup.find_all(class_='list-adress')
        listings_urls = [listing.find('a', href=True)['href'] for listing in listings]
        
        return listings_urls
    
    except Exception as e:
        return e

In [125]:
s = get_tor_session()
total_pages = get_number_of_pages('https://www.aruodas.lt/butu-nuoma/', s)

# Get listing urls for all of the pages. If an exception occurs while scraping, prints the exception.
listing_urls = []
for page_number in range(1, total_pages + 1):
    page_url = f'https://www.aruodas.lt/butu-nuoma/puslapis/{page_number}/?FOrder=AddDate&detailed_search=1'
    
    page_urls = get_page_urls(s, page_url)
    if type(page_results) == list:
        listing_urls.extend(page_urls)
    else:
        print(page_results)
        
# Save urls to a file? TODO: Is this necessary?
with open('../data/listing_urls.txt', 'w') as f:
    f.write('\n'.join(listing_urls))

Built a Tor session at IP 193.70.13.35.


In [259]:
def get_object_data(soup, html_tags=None):
    '''
    soup - BS4 object.
    
    TODO: Create env variable for html tags or save them to a file somehow.
    '''
    
    if html_tags is None:
        pass
    
    # Initialize HTML class, id, etc. tags.
    html_tag_listing_statistics = 'obj-top-stats'

    html_tag_building_energy_class = 'energy-class-tooltip'

    html_tag_object_details = ['obj-details', 'obj-details '] # Weird, but necessary.
    html_tag_object_details_names = 'dt'
    html_tag_object_details_items = 'dd'
    html_tag_object_description = 'collapsedText'
    html_tag_object_name = 'obj-header-text'

    html_tag_neighbourhood_statistics = 'advertStatisticHolder'
    html_tag_neighbourhood_statistics_names = 'cell-text'
    html_tag_neighbourhood_statistics_items = 'cell-data'

    html_tag_realtor = 'obj-contacts'
    html_tag_realtor_name = 'contacts-title'
    html_tag_no_realtor = 'obj-contacts simple'
    
    
    # Find all name and item classes withing object details class.
    object_details_class = soup.find(class_=html_tag_object_details[0])
    
#     Sometimes listings get displayed but have no info. Scraper cathers?
    if object_details_class is None:
        return None
        
    
    object_names = object_details_class.find_all(html_tag_object_details_names)
    object_items = object_details_class.find_all(html_tag_object_details_items)

    # Get field names.
    object_names = [item.contents[0] for item in object_names]

    # Extract values from item fields.
    object_item_values = []
    for object_item in object_items:

        # Single value items.
        if len(object_item) == 1:
            object_item_values.append(object_item.contents[0])

        # Multiple value items.
        elif len(object_item) > 1:

            # Tries to get list-like items.
            items = [item.contents[0] for item in object_item.find_all(class_='special-comma')]
            if len(items) != 0:
                object_item_values.append(items)

            # If the list is empty, the item is most likely single value (hence index 0), with additional ads, links, etc.
            else:
                object_item_values.append(object_item.contents[0])

    # Set everything as a dictionary.
    object_data = dict(zip(object_names, object_item_values))


    # Find all name and item classes withing object details class.
    neighbourhood_statistics = soup.find(id=html_tag_neighbourhood_statistics)

    neighbourhood_statistics_names = neighbourhood_statistics.find_all(class_=html_tag_neighbourhood_statistics_names)
    neighbourhood_statistics_items = neighbourhood_statistics.find_all(class_=html_tag_neighbourhood_statistics_items)

    # Extract the name and the item values, add them to the dictionary.
    neighbourhood_statistics_names = [item.contents[0] for item in neighbourhood_statistics_names]
    neighbourhood_statistics_items = [item.contents[0] for item in neighbourhood_statistics_items]

    for name, item in zip(neighbourhood_statistics_names, neighbourhood_statistics_items):
        object_data[name] = item

    # Get object listing statistics (views, favorites, etc.).
    listing_statistics = soup.find(class_=html_tag_listing_statistics).contents

    object_data[listing_statistics[0]] = listing_statistics[1].contents[0]
    
    try:
        object_data['Listing Favorites'] = listing_statistics[3].contents[0]
    except:
        object_data['Listing Favorites'] = 0

    # Get object description text.
    object_data['Object Description'] = soup.find(id=html_tag_object_description).contents

    # Get object name - City, Neighbourhood, Street, Other.
    object_data['Listing Name'] = soup.find(class_=html_tag_object_name).contents[0]

    # Get energy class rating.
    building_energy_class = soup.find(class_=html_tag_building_energy_class)

    try:
        object_data['Building Energy Class'] = building_energy_class.contents[1]
        object_data['Building Energy Class Category'] = building_energy_class.contents[2]
    except:
        object_data['Building Energy Class'] = np.nan
        object_data['Building Energy Class Category'] = np.nan

    # Check if a realtor is present, get it's name if it is.
    object_contacts = soup.find(class_=html_tag_no_realtor)
    if object_contacts is None:
        realtor_name = soup.find(class_=html_tag_realtor).find(class_=html_tag_realtor_name).contents[0]
        # Handles cases where the realtor is a company, not a person. TODO: Add RealtorCompany variable.
        if realtor_name == 'Pardavėjo kontaktai':
            object_data['Realtor Name'] = np.nan
            object_data['Realtor'] = False
        else:
            object_data['Realtor Name'] = realtor_name
            object_data['Realtor'] = True
    else:
        object_data['Realtor Name'] = np.nan
        object_data['Realtor'] = False

    return object_data

In [287]:
def parse_object_data(object_data):
    '''
    TODO: Descr.
    '''
    variables_integer = ['Plotas', 'KainaMėn', 'KambariųSk', 'Aukštas', 'AukštųSk', 'Metai', 'ArtimiausiasDarželis', 'ArtimiausiaMokymoĮstaiga', 'ArtimiausiaParduotuvė', 'ViešojoTransportoStotelė', 'Nusikaltimai500MSpinduliuPraėjusįMėnesį']
    variables_categorical = ['PastatoTipas', 'Šildymas', 'Įrengimas']
    variables_lists = ['Ypatybės', 'PapildomosPatalpos', 'PapildomaĮranga', 'Apsauga']
    
    
    # Transform keys from whatever messy format to VariableName.
    keys = list(object_data.keys())
    keys = [key.strip() for key in keys]
    keys = [key.translate(str.maketrans('', '', string.punctuation)) for key in keys]
    keys = [''.join(key.lower().title().split(' ')) for key in keys]

    object_data = dict(zip(keys, list(object_data.values())))
    
    
    # Convert integer variables to integer.
    for variable in variables_integer:
        # Handles cases when variable is missing:
        try:
            object_data[variable] = int(re.findall('\d+', object_data[variable])[0])
        except:
            continue

    # Remove whitespace from categorical variables.
    for variable in variables_categorical:
        object_data[variable] = object_data[variable].strip()


    # Transform lists of variables into pseudo categorical variables. E.g. Variable: ['feat1', 'feat2'] -> Variable_feat1: 1, Variable_feat2: 1.
    for variable in variables_lists:
        # Convert space delimited text to TitleCamelCase. E.g. 'This house' -> 'ThisHouse'.
        object_data[variable] = [''.join(item.title().split(' ')) for item in object_data[variable]]

        # Create a pseudo categorical variable where multiple values in a category is present.
        for feature in object_data[variable]:
            object_data[variable + '_' + feature] = 1 


    # Split ListingName into City, Neighbourhood, Street.
    object_data['ListingName'] = object_data['ListingName'].split(',')
    object_data['BuildingCity'] = object_data['ListingName'][0].strip()
    object_data['BuildingNeighbourhood'] = object_data['ListingName'][1].strip()
    object_data['BuildingStreet'] = object_data['ListingName'][2].strip()


    # Split Total/Today views into separate variables.
    object_data['SkelbimąPeržiūrėjoIšVisošiandien'] = object_data['SkelbimąPeržiūrėjoIšVisošiandien'].split('/')
    object_data['ListingViewsTotal'] = int(object_data['SkelbimąPeržiūrėjoIšVisošiandien'][0])
    object_data['ListingViewsToday'] = int(object_data['SkelbimąPeržiūrėjoIšVisošiandien'][1])


    # Transform ObjectDescription into a single string, as well as replacing <br/>'s with \n.
    object_data['ObjectDescription'] = ''.join([str(item) for item in object_data['ObjectDescription']])
    object_data['ObjectDescription'] = re.sub('<br/s*?>', '\n', object_data['ObjectDescription'])


    # Drop no longer required variables.
    variables_drop = []
    variables_drop.extend(variables_lists)
    variables_drop.extend(['ListingName', 'SkelbimąPeržiūrėjoIšVisošiandien'])
    for variable in variables_drop:
        object_data.pop(variable)
    
    
    return object_data

In [290]:
with open('../data/listing_urls.txt', 'r') as f:
    listing_urls = f.readlines()
    
res = []
try:
    for listing_url in listing_urls[1:5]:
        driver = webdriver.Chrome('D:/Projects/Personal/Current/Rent AVM/rent-avm/scraper/chromedriver.exe')

        driver.get(listing_url)
        page_source = driver.page_source
        driver.close()

        soup = BeautifulSoup(page_source, 'lxml')

        res.append(get_object_data(soup))
except:
    pass
    

df = pd.DataFrame()
for item in res:
    df = df.append(pd.DataFrame(parse_object_data(item), index=[0]))
    
pd.set_option('display.max_columns', 500)
df

,Plotas,KainaMėn,KambariųSk,Aukštas,AukštųSk,Metai,PastatoTipas,Šildymas,Įrengimas,ArtimiausiasDarželis,ArtimiausiaMokymoĮstaiga,ArtimiausiaParduotuvė,ViešojoTransportoStotelė,Nusikaltimai500MSpinduliuPraėjusįMėnesį,ListingFavorites,ObjectDescription,BuildingEnergyClass,BuildingEnergyClassCategory,RealtorName,Realtor,Ypatybės_UždarasKiemas,Ypatybės_Internetas,Ypatybės_AukštosLubos,Ypatybės_NaujaKanalizacija,Ypatybės_NaujaElektrosInstaliacija,Ypatybės_VirtuvėSujungtaSuKambariu,PapildomosPatalpos_Balkonas,PapildomosPatalpos_VietaAutomobiliui,PapildomaĮranga_SuBaldais,PapildomaĮranga_Kondicionierius,PapildomaĮranga_SkalbimoMašina,PapildomaĮranga_Šaldytuvas,PapildomaĮranga_VirtuvėsKomplektas,PapildomaĮranga_Viryklė,PapildomaĮranga_Indaplovė,PapildomaĮranga_Vonia,Apsauga_ŠarvuotosDurys,Apsauga_Signalizacija,Apsauga_KodinėLaiptinėsSpyna,BuildingCity,BuildingNeighbourhood,BuildingStreet,ListingViewsTotal,ListingViewsToday,PapildomosPatalpos_Drabužinė,PapildomaĮranga_ŠildomosGrindys,PapildomaĮranga_DušoKabina,Apsauga_Videokameros,Apsauga_BudintisSargas,NamoNumeris,ButoNumeris,VidutiniškaiTiekKainuotųŠildymas1Mėn,Ypatybės_TualetasIrVoniaAtskirai,Ypatybės_KabelinėTelevizija,PapildomosPatalpos_Sandėliukas,PapildomosPatalpos_Terasa,PapildomosPatalpos_Rūsys,PapildomaĮranga_PlastikiniaiVamzdžiai,Ypatybės_ButasPalėpėje,PapildomosPatalpos_YraPalėpė
0,110,990,3,2,2,2005,Mūrinis,"Centrinis kolektorinis, centrinis",Įrengtas,210.0,190.0,270.0,200.0,0.0,0,\n Išskirtinis butas prestižinė...,NaN,NaN,NaN,False,1,1,1.0,1.0,1.0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,1,1.0,1.0,Vilnius,Senamiestis,Dysnos g.,36,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,103,899,3,11,27,2004,Monolitinis,Centrinis kolektorinis,Įrengtas,NaN,NaN,NaN,NaN,NaN,0,\n -\tJOKIŲ TARPININKAVIMO MOKE...,NaN,NaN,Akorus Real Estate,True,1,1,NaN,1.0,1.0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,1,1.0,1.0,Vilnius,Šnipiškės,Konstitucijos pr.,48,48,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,113,1,4,7,9,2010,Mūrinis,Centrinis kolektorinis,Įrengtas,290.0,130.0,130.0,NaN,0.0,6,\n Išnuomosiu 4 kambarių 113 kv...,NaN,NaN,NaN,False,1,1,NaN,1.0,1.0,1,1.0,1.0,1,1.0,1,1,1,1,1,1,1,1.0,1.0,Kaunas,Centras,K. Donelaičio g.,5195,122,1.0,NaN,1.0,1.0,NaN,\n 81A ...,\n 51 ...,"\n 19,21 €/mėn.",1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
0,54,500,1,3,3,1940,Mūrinis,Dujinis,Įrengtas,100.0,140.0,190.0,170.0,0.0,2,"\n Erdvus, saulėtas, nestadarti...",NaN,NaN,NaN,False,1,1,1.0,NaN,NaN,1,NaN,NaN,1,NaN,1,1,1,1,1,1,1,NaN,NaN,Vilnius,Senamiestis,Šv. Stepono g.,732,732,NaN,1.0,NaN,NaN,NaN,\n 9 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [223]:


soup.find(class_=class_tag_object_details).find_all(class_tag_object_details_items)

# Some of these are Optional!
# order_object_details = {
#     'ObjectArea': 0,
#     'ObjectPrice': 1,
#     'ObjectNumberOfRooms': 2,
#     'objectFloorNumber': 3,
#     'BuildingFloorsTotal': 4,
#     'BuildingBuildYear': 5,
#     'BuildingType': 6,
#     'BuildingHeating': 7,
#     'BuildingStatus': 8,
#     # Features below have more than one item.
#     'ObjectFeatures': 9,
#     'ObjectExtraAreas': 10,
#     'ObjectUtilities': 11,
#     'ObjectSecurity': 12
# }



[<dd>
                         48 m²                                            </dd>,
 <dd>
                         390 €                                            </dd>,
 <dd>
                         2                                            </dd>,
 <dd>
                         5                                            </dd>,
 <dd>
                         9                                            </dd>,
 <dd>
                         1996                                            </dd>,
 <dd>
                         Mūrinis                                            </dd>,
 <dd>
                         Centrinis                                            </dd>,
 <dd>
                         Įrengtas                                                <div class="img-usefull">NAUDINGA:</div>
 <a href="/interjeras/?utm_source=aruodas&amp;utm_medium=skelbime&amp;utm_campaign=interjeras" style="color: #0099d3; border-bottom:1px dashed #0099d3; margin: 0 2px; text-decoration:

In [225]:
soup.find(class_=class_tag_object_details).find_all(class_tag_object_details_items)

[<dd>
                         48 m²                                            </dd>,
 <dd>
                         390 €                                            </dd>,
 <dd>
                         2                                            </dd>,
 <dd>
                         5                                            </dd>,
 <dd>
                         9                                            </dd>,
 <dd>
                         1996                                            </dd>,
 <dd>
                         Mūrinis                                            </dd>,
 <dd>
                         Centrinis                                            </dd>,
 <dd>
                         Įrengtas                                                <div class="img-usefull">NAUDINGA:</div>
 <a href="/interjeras/?utm_source=aruodas&amp;utm_medium=skelbime&amp;utm_campaign=interjeras" style="color: #0099d3; border-bottom:1px dashed #0099d3; margin: 0 2px; text-decoration:

In [196]:
, str(a.contents))

<re.Match object; span=(24, 27), match='390'>

In [166]:
soup

<!DOCTYPE html>

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"278f11442b",applicationID:"43157175"};window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var i=n[t]={exports:{}};e[t][0].call(i.exports,function(n){var i=e[t][1][n];return r(i||n)},i,i.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<t.length;i++)r(t[i]);return r}({1:[function(e,n,t){function r(){}function i(e,n,t){return function(){return o(e,[u.now()].concat(f(arguments)),n?null:this,t),n?void 0:this}}var o=e("handle"),a=e(4),f=e(5),c=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],l="api-",d=l+"ixn-";a(p,function(e,n){s[n]=i(l+n,!0,"api")}),s.addPageAction=i(l+"addPageAc

['https://www.aruodas.lt/butai-vilniuje-pasilaiciuose-laisves-pr-4-27811035/',
 'https://www.aruodas.lt/butu-nuoma-klaipedoje-centre-j-janonio-g-isnuomojamas-tvarkingas-2-kambariu-butas-4-986385/',
 'https://www.aruodas.lt/butu-nuoma-vilniuje-zveryne-blindziu-g-nuomojamas-kokybiskai-irengtas-butas-zveryne-4-986383/',
 'https://www.aruodas.lt/butu-nuoma-vilniuje-senamiestyje-m-dauksos-g-4-986381/',
 'https://www.aruodas.lt/butu-nuoma-vilniuje-pasilaiciuose-eitminu-g-nuomojamas-erdvus-naujai-irengtas-studijos-4-986379/',
 'https://www.aruodas.lt/butu-nuoma-kaune-dainavoje-partizanu-g-nuomuojamas-jaukus-erdvus-dvieju-kambariu-4-986377/',
 'https://www.aruodas.lt/butu-nuoma-klaipedoje-centre-medziotoju-g-isnuomojamas-3-kambariu-butas-medziotoju-g-4-986375/',
 'https://www.aruodas.lt/butu-nuoma-vilniuje-fabijoniskese-s-staneviciaus-g-be-tarpininku-ilgalaikei-nuomai-nuo-202006-4-937307/',
 'https://www.aruodas.lt/butu-nuoma-vilniuje-naujamiestyje-gelezinio-vilko-g-nuomojamas-jaukus-bei-tvark

In [113]:
[item for item in listing.find(class_='list-adress')]

['\n', <h3>
 <a href="https://www.aruodas.lt/butu-nuoma-kaune-dainavoje-partizanu-g-nuomuojamas-jaukus-erdvus-dvieju-kambariu-4-986377/">Kaunas, Dainava<br>Partizanų g.</br></a>
 </h3>, '\n', <p class="flat-rent-dateadd">Prieš 14 min.</p>, '\n', <div class="price">
 <span class="list-item-price">125 €</span>
 <span class="price-pm">
                                                                                                             2,72                                        €/m²                                                                                                </span>
 </div>, '\n']

In [106]:
type(listing.find_all(class_='list-adress'))#, href=True)#.get('h3')

bs4.element.ResultSet

In [99]:
[child for child in a.children]

['\n', <h3>
 <a href="https://www.aruodas.lt/butu-nuoma-kaune-dainavoje-partizanu-g-nuomuojamas-jaukus-erdvus-dvieju-kambariu-4-986377/">Kaunas, Dainava<br>Partizanų g.</br></a>
 </h3>, '\n', <p class="flat-rent-dateadd">Prieš 14 min.</p>, '\n', <div class="price">
 <span class="list-item-price">125 €</span>
 <span class="price-pm">
                                                                                                             2,72                                        €/m²                                                                                                </span>
 </div>, '\n']

In [61]:
listings[5]

<tr class="list-row ">
<td class="list-img"><!--
--><div class="list-photo">
<a href="/butu-nuoma-kaune-dainavoje-partizanu-g-nuomuojamas-jaukus-erdvus-dvieju-kambariu-4-986377/">
<img alt="Kaunas, Dainava, Partizanų g., 2 kambarių buto nuoma" data-default="https://skelbiu-img.dgn.lt/1_19_655462561/nuomuojamas-jaukus-erdvus-dvieju-kambariu-butas.jpg" data-extra="https://skelbiu-img.dgn.lt/1_21_655462569/nuomuojamas-jaukus-erdvus-dvieju-kambariu-butas.jpg,https://skelbiu-img.dgn.lt/1_21_655462559/nuomuojamas-jaukus-erdvus-dvieju-kambariu-butas.jpg,https://skelbiu-img.dgn.lt/1_21_655462575/nuomuojamas-jaukus-erdvus-dvieju-kambariu-butas.jpg" src="https://skelbiu-img.dgn.lt/1_19_655462561/nuomuojamas-jaukus-erdvus-dvieju-kambariu-butas.jpg" title="Kaunas, Dainava, Partizanų g., 2 kambarių buto nuoma | Aruodas.lt"/>
</a>
<div class="icon-on-top-container">
</div>
</div><!--
--><div class="icon-house-list flout-icon">
<div class="house-count">1</div>
<div class="simple-info-tooltip">Skelbim

In [7]:
soup

<!DOCTYPE html>

<html>
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="telephone=no" name="format-detection">
<title>Butai nuomai Nekilnojamo turto skelbimai - Aruodas.lt</title>
<meta content="Butai nuomai. Skelbimai, kuriuose nuomojami butai - Aruodas.lt" name="description"/>
<link href="https://aruodas-static.dgn.lt/static/img/favicon.ico" rel="shortcut icon"/>
<meta content="52ac462816d417ec3c57b60b91ee3631" name="verify-paysera">
<meta content="1673212931" property="fb:admins"/>
<meta content="1076122818" property="fb:admins"/>
<meta content="523544227" property="fb:admins"/>
<meta content="1445021322" property="fb:admins"/>
<meta content="https://aruodas-img.dgn.lt/object_64_88576559/nuotrauka.jpg" property="og:image"/>
<meta content="Butai nuomai" property="og:title"/>
<meta content="Aruodas.lt" property="og:site_name"/>
<meta content="website" property="og:type"/>
<meta content="https://www.aruodas.lt/butu-nuoma/" property="og:url"/>
